In [9]:
import pandas as pd
from collections import defaultdict
import numpy as np

def ngrams(text, n):
    return [' '.join(text[i:i+n]) for i in range(len(text)-n+1)]

def explode_str(df, col, sep, n):
    s = df[col].str.split(sep)
    i = np.arange(len(s)).repeat(s.apply(lambda x: max(len(x) - n + 1, 0)))
    ngrams_list = s.apply(lambda x: ngrams(x, n)).explode().reset_index(drop=True)
    return df.iloc[i].assign(**{col: ngrams_list})



In [10]:
from google.colab import files
uploaded = files.upload()

Saving Bing search terms.csv to Bing search terms (1).csv


In [11]:
# queries.csv contains your search term report with KPIs
# Adjust delimiter by setting the sep= parameter
print("----------------------------------------------------------------------------------------------------")
print("\tThe raw report")
print("----------------------------------------------------------------------------------------------------")
df = pd.read_csv("Bing search terms.csv")
print(df)

----------------------------------------------------------------------------------------------------
	The raw report
----------------------------------------------------------------------------------------------------
                                                 Query  Impressions  Clicks  \
0        air conduction domes for enjoyee hearing aids         1000     341   
1                           choosing hearing aid domes         1010     351   
2    do hearing aid open domes come in different sizes         1020     361   
3                               domes for hearing aids         1030     371   
4                 ear plastic inserts for hearing aids         1040     381   
..                                                 ...          ...     ...   
398                widex hearing aid replacement tubes         4980    4321   
399              hearclear hc64 high power hearing aid         4990    4331   
400                         hear clear go hearing aids         5000    

In [14]:
# Create 1-Grams by splitting 1 row to multiple rows
# "Query" is the columnname where you want to apply the n-Gram analysis
df_1grams = explode_str(df, 'Query', ' ', 1)
print("")
print("----------------------------------------------------------------------------------------------------")
print("\tSplitting into 1-Grams")
print("----------------------------------------------------------------------------------------------------")
df_1grams.tail()



----------------------------------------------------------------------------------------------------
	Splitting into 1-Grams
----------------------------------------------------------------------------------------------------


,Query,Impressions,Clicks,Costs,Conversions,ConversionValue
402,affordale,5020,4361,413,103,40700
402,affordale,5020,4361,413,103,40700
402,affordale,5020,4361,413,103,40700
402,affordale,5020,4361,413,103,40700
402,affordale,5020,4361,413,103,40700


In [15]:
# Create 2-Grams by splitting 1 row to multiple rows
df_2grams = explode_str(df, 'Query', ' ', 2)
print("")
print("----------------------------------------------------------------------------------------------------")
print("\tSplitting into 2-Grams")
print("----------------------------------------------------------------------------------------------------")
print(df_2grams)


----------------------------------------------------------------------------------------------------
	Splitting into 2-Grams
----------------------------------------------------------------------------------------------------
              Query  Impressions  Clicks  Costs  Conversions  ConversionValue
0    air conduction         1000     341     11            1              500
0    air conduction         1000     341     11            1              500
0    air conduction         1000     341     11            1              500
0    air conduction         1000     341     11            1              500
0    air conduction         1000     341     11            1              500
..              ...          ...     ...    ...          ...              ...
402       aid tubes         5020    4361    413          103            40700
402       aid tubes         5020    4361    413          103            40700
402       aid tubes         5020    4361    413          103           

In [16]:
# Group by N-Grams
df_1grams = df_1grams.groupby('Query').sum()
df_1grams = df_1grams.reset_index()
print("")
print("----------------------------------------------------------------------------------------------------")
print("\tAggregated Metrics on 1-Grams")
print("----------------------------------------------------------------------------------------------------")
print(df_1grams)


----------------------------------------------------------------------------------------------------
	Aggregated Metrics on 1-Grams
----------------------------------------------------------------------------------------------------
     Query  Impressions  Clicks  Costs  Conversions  ConversionValue
0        -         8280    6962    650          162            63800
1     11mm        39380   30154   2692          659           260800
2       12        19740   15786   1440          354           140400
3      200        10140    6186    480          114            44400
4      8mm        43730   33845   3038          746           294800
..     ...          ...     ...    ...          ...              ...
115   what        62940   52396   4870         1194           477400
116  which        24500   21205   2005          500           197500
117   with        17700   14405   1325          330           129500
118     x1        27160   22547   2093          511           205100
119    

In [17]:
df_2grams = df_2grams.groupby('Query').sum()
df_2grams = df_2grams.reset_index()
print("")
print("----------------------------------------------------------------------------------------------------")
print("\tAggregated Metrics on 2-Grams")
print("----------------------------------------------------------------------------------------------------")
print(df_2grams)


----------------------------------------------------------------------------------------------------
	Aggregated Metrics on 2-Grams
----------------------------------------------------------------------------------------------------
            Query  Impressions  Clicks  Costs  Conversions  ConversionValue
0         - sizes        14120   11484   1056          260           103200
1    11mm hearing        12960    9665    851          204            82100
2           12 mm         8490    6513    582          144            56400
3          13 trs        28500   24546   2316          576           228000
4     200 hearing         6240    3604    268           60            24400
..            ...          ...     ...    ...          ...              ...
203     what size        20250   16955   1580          390           155000
204    which dome         4140    3481    325           81            31900
205       with my        57760   45239   4085          988           397100
206  x

In [18]:
print("")
print("----------------------------------------------------------------------------------------------------")
print("\tAdded Calculated Measures")
print("----------------------------------------------------------------------------------------------------")


----------------------------------------------------------------------------------------------------
	Added Calculated Measures
----------------------------------------------------------------------------------------------------


In [19]:
# Define your calculated measures here:
df_1grams['CR'] = df_1grams['Conversions']/df_1grams['Clicks']
df_1grams['CPO'] = df_1grams['Costs'] / df_1grams['Conversions']
print("1-Grams:")
print(df_1grams)

1-Grams:
     Query  Impressions  Clicks  Costs  Conversions  ConversionValue  \
0        -         8280    6962    650          162            63800   
1     11mm        39380   30154   2692          659           260800   
2       12        19740   15786   1440          354           140400   
3      200        10140    6186    480          114            44400   
4      8mm        43730   33845   3038          746           294800   
..     ...          ...     ...    ...          ...              ...   
115   what        62940   52396   4870         1194           477400   
116  which        24500   21205   2005          500           197500   
117   with        17700   14405   1325          330           129500   
118     x1        27160   22547   2093          511           205100   
119    you         9600    6305    515          120            48500   

           CR       CPO  
0    0.023269  4.012346  
1    0.021854  4.084977  
2    0.022425  4.067797  
3    0.018429  4.21052

In [20]:
df_2grams['CR'] = df_2grams['Conversions']/df_2grams['Clicks']
df_2grams['CPO'] = df_2grams['Costs'] / df_2grams['Conversions']
print("\n2-Grams:")
print(df_2grams)


2-Grams:
            Query  Impressions  Clicks  Costs  Conversions  ConversionValue  \
0         - sizes        14120   11484   1056          260           103200   
1    11mm hearing        12960    9665    851          204            82100   
2           12 mm         8490    6513    582          144            56400   
3          13 trs        28500   24546   2316          576           228000   
4     200 hearing         6240    3604    268           60            24400   
..            ...          ...     ...    ...          ...              ...   
203     what size        20250   16955   1580          390           155000   
204    which dome         4140    3481    325           81            31900   
205       with my        57760   45239   4085          988           397100   
206  x1 invisible        13240   10604    968          240            94400   
207       you buy         5220    3243    255           63            23700   

           CR       CPO  
0    0.022640  

In [21]:
df_1grams.to_csv("1grams.csv",index=False)
df_2grams.to_csv("2grams.csv",index=False)
files.download('1grams.csv')
files.download('2grams.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>